In [68]:
# %%
# 0. Imports & Spark session

import os

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import DecimalType

from dotenv import load_dotenv

load_dotenv()

# ADLS config
storage_account = "strealestatehamza"
container = "realestate"

adls_key = os.getenv("ADLS_ACCOUNT_KEY")
if not adls_key:
    raise RuntimeError("ADLS_ACCOUNT_KEY missing from .env")

spark = (
    SparkSession.builder
    .appName("Bronze_Avito_Mubawab")
    .getOrCreate()
)

# ADLS auth
spark.conf.set(
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
    adls_key,
)
spark.conf.set(
    f"fs.azure.account.key.{storage_account}.blob.core.windows.net",
    adls_key,
)

raw_base = f"abfss://{container}@{storage_account}.dfs.core.windows.net/raw"
bronze_base = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze"

spark


In [69]:
# %%
# Helper: read parquet only if the path pattern matches at least one file

from pyspark.sql.utils import AnalysisException

def read_parquet_if_exists(path_pattern: str):
    """
    Try to read parquet for a glob pattern.
    Return a DataFrame or None if nothing exists.
    """
    try:
        df = spark.read.parquet(path_pattern)
        print(f"📂 Reading: {path_pattern}")
        return df
    except AnalysisException as e:
        msg = str(e)
        if "Path does not exist" in msg or "PATH_NOT_FOUND" in msg:
            print(f"⚠️ No data found for: {path_pattern}")
            return None
        # if it's some other error, re-raise
        raise


In [70]:
# %%
# 1. Read RAW Avito (ventes + locations) - NEW LAYOUT WITH page=* FOLDER

avito_ventes_path = f"{raw_base}/avito/ventes/*/*/*/*/page=*"
avito_locations_path = f"{raw_base}/avito/locations/*/*/*/*/page=*"

avito_ventes_df = read_parquet_if_exists(avito_ventes_path)
avito_locations_df = read_parquet_if_exists(avito_locations_path)

avito_dfs = [df for df in [avito_ventes_df, avito_locations_df] if df is not None]

if not avito_dfs:
    raise RuntimeError("No Avito raw data found (ventes or locations).")

if len(avito_dfs) == 1:
    df_avito = avito_dfs[0]
else:
    df_avito = avito_dfs[0].unionByName(avito_dfs[1])

df_avito.printSchema()

📂 Reading: abfss://realestate@strealestatehamza.dfs.core.windows.net/raw/avito/ventes/*/*/*/*/page=*
⚠️ No data found for: abfss://realestate@strealestatehamza.dfs.core.windows.net/raw/avito/locations/*/*/*/*/page=*
root
 |-- attributes: string (nullable = true)
 |-- breadcrumbs: string (nullable = true)
 |-- breadcrumbs_list: string (nullable = true)
 |-- category_label: string (nullable = true)
 |-- description: string (nullable = true)
 |-- equipments: string (nullable = true)
 |-- id: string (nullable = true)
 |-- images: string (nullable = true)
 |-- location: string (nullable = true)
 |-- price_text: string (nullable = true)
 |-- published_date: string (nullable = true)
 |-- scraping_time: string (nullable = true)
 |-- seller_is_store: boolean (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- seller_url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- source_site: string (nullable = true)
 |-- offre: string (nu

In [71]:
# %%
# 2. Read RAW Mubawab (ventes + locations) - NEW LAYOUT WITH page=* FOLDER

mubawab_ventes_path = f"{raw_base}/mubawab/ventes/*/*/*/*/page=*"
mubawab_locations_path = f"{raw_base}/mubawab/locations/*/*/*/*/page=*"

mubawab_ventes_df = read_parquet_if_exists(mubawab_ventes_path)
mubawab_locations_df = read_parquet_if_exists(mubawab_locations_path)

mubawab_dfs = [df for df in [mubawab_ventes_df, mubawab_locations_df] if df is not None]

if not mubawab_dfs:
    raise RuntimeError("No Mubawab raw data found (ventes or locations).")

if len(mubawab_dfs) == 1:
    df_mubawab = mubawab_dfs[0]
else:
    df_mubawab = mubawab_dfs[0].unionByName(mubawab_dfs[1])

df_mubawab.printSchema()

📂 Reading: abfss://realestate@strealestatehamza.dfs.core.windows.net/raw/mubawab/ventes/*/*/*/*/page=*
⚠️ No data found for: abfss://realestate@strealestatehamza.dfs.core.windows.net/raw/mubawab/locations/*/*/*/*/page=*
root
 |-- attributes: string (nullable = true)
 |-- breadcrumbs: string (nullable = true)
 |-- breadcrumbs_list: string (nullable = true)
 |-- category_label: string (nullable = true)
 |-- description: string (nullable = true)
 |-- equipments: string (nullable = true)
 |-- id: string (nullable = true)
 |-- images: string (nullable = true)
 |-- location: string (nullable = true)
 |-- price_text: string (nullable = true)
 |-- published_date: string (nullable = true)
 |-- scraping_time: string (nullable = true)
 |-- seller_is_store: boolean (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- seller_url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- source_site: string (nullable = true)
 |-- offre: string

In [72]:
# %% 
# BRONZE Avito

bronze_avito = df_avito.select(
    # Identifiers
    "id",
    "url",
    "source_site",
    "offre",

    # Main business content
    "title",
    "description",
    "price_text",
    "location",

    # Dates
    "published_date",
    "scraping_time",
    "ingest_ts",

    # Metadata
    "category_label",
    "breadcrumbs",
    "breadcrumbs_list",
    "attributes",

    # Media
    "images",
    "equipments",

    # Seller info
    "seller_name",
    "seller_url",
    "seller_is_store"
)

bronze_avito.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- source_site: string (nullable = true)
 |-- offre: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- price_text: string (nullable = true)
 |-- location: string (nullable = true)
 |-- published_date: string (nullable = true)
 |-- scraping_time: string (nullable = true)
 |-- ingest_ts: string (nullable = true)
 |-- category_label: string (nullable = true)
 |-- breadcrumbs: string (nullable = true)
 |-- breadcrumbs_list: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- images: string (nullable = true)
 |-- equipments: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- seller_url: string (nullable = true)
 |-- seller_is_store: boolean (nullable = true)



In [73]:
# %%
# BRONZE Mubawab

bronze_mubawab = df_mubawab.select(
    # Identifiers
    "id",
    "url",
    "source_site",
    "offre",

    # Main business content
    "title",
    "description",
    "price_text",
    "location",

    # Dates
    "published_date",
    "scraping_time",
    "ingest_ts",

    # Metadata
    "category_label",
    "breadcrumbs",
    "breadcrumbs_list",
    "attributes",

    # Media
    "images",
    "equipments",

    # Seller info
    "seller_name",
    "seller_url",
    "seller_is_store"
)

bronze_mubawab.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- source_site: string (nullable = true)
 |-- offre: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- price_text: string (nullable = true)
 |-- location: string (nullable = true)
 |-- published_date: string (nullable = true)
 |-- scraping_time: string (nullable = true)
 |-- ingest_ts: string (nullable = true)
 |-- category_label: string (nullable = true)
 |-- breadcrumbs: string (nullable = true)
 |-- breadcrumbs_list: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- images: string (nullable = true)
 |-- equipments: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- seller_url: string (nullable = true)
 |-- seller_is_store: boolean (nullable = true)



In [74]:
# %%
# NULL id / url

null_count_avito = bronze_avito.filter(
    (F.col("id").isNull()) | (F.col("url").isNull())
).count()

null_count_mubawab = bronze_mubawab.filter(
    (F.col("id").isNull()) | (F.col("url").isNull())
).count()

print("NULL id/url Avito:", null_count_avito)
print("NULL id/url Mubawab:", null_count_mubawab)

NULL id/url Avito: 0
NULL id/url Mubawab: 0


In [75]:
# %%
# Doublons par URL (avant dédoublonnage)

bronze_avito.groupBy("url").count().filter("count > 1").show(20, truncate=False)
bronze_mubawab.groupBy("url").count().filter("count > 1").show(20, truncate=False)

+---+-----+
|url|count|
+---+-----+
+---+-----+

+----------------------------------------------------------------------------------------------------------------------------+-----+
|url                                                                                                                         |count|
+----------------------------------------------------------------------------------------------------------------------------+-----+
|https://www.mubawab.ma/fr/a/8257658/%C3%A0-vendre-%E2%80%93-bel-appartement-spacieux-ferme-bretonne                         |2    |
|https://www.mubawab.ma/fr/a/8257646/emplacement-strat%C3%A9gique-appartement-%C3%A0-vendre                                  |2    |
|https://www.mubawab.ma/fr/a/8257668/villa-a-vendre                                                                          |2    |
|https://www.mubawab.ma/fr/a/8257591/vente-appartement-haut-standing-moujahidine                                             |2    |
|https://www.mubawab

In [76]:
# %%
w_avito = Window.partitionBy("url").orderBy(F.col("scraping_time").asc())

bronze_avito = (
    bronze_avito
    .withColumn("rn", F.row_number().over(w_avito))
    .filter("rn = 1")
    .drop("rn")
)

w_mubawab = Window.partitionBy("url").orderBy(F.col("scraping_time").asc())

bronze_mubawab = (
    bronze_mubawab
    .withColumn("rn", F.row_number().over(w_mubawab))
    .filter("rn = 1")
    .drop("rn")
)

In [77]:
# %%
# Vérif (il ne doit plus y avoir de doublons)

bronze_avito.groupBy("url").count().filter("count > 1").show(10, truncate=False)
bronze_mubawab.groupBy("url").count().filter("count > 1").show(10, truncate=False)

+---+-----+
|url|count|
+---+-----+
+---+-----+

+---+-----+
|url|count|
+---+-----+
+---+-----+



In [78]:
# %%
bronze_avito.groupBy("offre").count().orderBy("count", ascending=False).show(truncate=False)
bronze_mubawab.groupBy("offre").count().orderBy("count", ascending=False).show(truncate=False)

+-----+-----+
|offre|count|
+-----+-----+
|vente|2797 |
+-----+-----+

+-----+-----+
|offre|count|
+-----+-----+
|vente|1175 |
+-----+-----+



In [79]:
# %%
bronze_avito.select("price_text").show(20, truncate=False)
bronze_mubawab.select("price_text").show(20, truncate=False)

+------------+
|price_text  |
+------------+
|2 335 000 DH|
|1 700 000 DH|
|1 500 000 DH|
|            |
|1 540 000 DH|
|2 200 000 DH|
|1 470 000 DH|
|1 850 000 DH|
|1 950 000 DH|
|1 350 000 DH|
|2 250 000 DH|
|            |
|1 800 000 DH|
|1 600 000 DH|
|1 812 000 DH|
|1 600 000 DH|
|1 180 000 DH|
|1 980 000 DH|
|1 850 000 DH|
|830 000 DH  |
+------------+
only showing top 20 rows

+----------------+
|price_text      |
+----------------+
|800 000 DH      |
|480 000 EUR     |
|Prix à consulter|
|Prix à consulter|
|Prix à consulter|
|820 000 DH      |
|Prix à consulter|
|2 100 000 DH    |
|1 000 000 DH    |
|Prix à consulter|
|Prix à consulter|
|1 450 000 DH    |
|460 000 DH      |
|500 000 DH      |
|280 000 EUR     |
|600 000 DH      |
|2 200 000 DH    |
|1 240 000 DH    |
|3 000 000 DH    |
|4 800 000 DH    |
+----------------+
only showing top 20 rows



In [80]:
# %%
# Avito: price_text -> price (double)

bronze_avito = (
    bronze_avito
    .withColumn(
        "price",
        F.when(
            (F.col("price_text").isNull()) |
            (F.lower(F.col("price_text")) == "null") |
            (F.trim(F.col("price_text")) == ""),
            None
        ).otherwise(
            F.regexp_replace(
                F.regexp_replace(
                    F.col("price_text"),
                    r"(?i)\s*dh\s*", ""     # remove DH / dh
                ),
                r"[^\d]", ""               # keep only digits
            ).cast("double")
        )
    )
    .drop("price_text")
)

bronze_avito.select("price").show(20, truncate=False)

+---------+
|price    |
+---------+
|2335000.0|
|1700000.0|
|1500000.0|
|NULL     |
|1540000.0|
|2200000.0|
|1470000.0|
|1850000.0|
|1950000.0|
|1350000.0|
|2250000.0|
|NULL     |
|1800000.0|
|1600000.0|
|1812000.0|
|1600000.0|
|1180000.0|
|1980000.0|
|1850000.0|
|830000.0 |
+---------+
only showing top 20 rows



In [81]:
# %%
# Mubawab: price_text -> price (double)

bronze_mubawab = (
    bronze_mubawab
    .withColumn(
        "price",
        F.when(
            (F.col("price_text").isNull()) |
            (F.lower(F.col("price_text")) == "null") |
            (F.trim(F.col("price_text")) == ""),
            None
        ).otherwise(
            F.regexp_replace(
                F.regexp_replace(
                    F.col("price_text"),
                    r"(?i)\s*dh\s*", ""
                ),
                r"[^\d]", ""
            ).cast("double")
        )
    )
    .drop("price_text")
)

bronze_mubawab.select("price").show(20, truncate=False)


+---------+
|price    |
+---------+
|800000.0 |
|480000.0 |
|NULL     |
|NULL     |
|NULL     |
|820000.0 |
|NULL     |
|2100000.0|
|1000000.0|
|NULL     |
|NULL     |
|1450000.0|
|460000.0 |
|500000.0 |
|280000.0 |
|600000.0 |
|2200000.0|
|1240000.0|
|3000000.0|
|4800000.0|
+---------+
only showing top 20 rows



In [82]:
# %%
# Cast en DECIMAL(20,0)

bronze_avito = bronze_avito.withColumn(
    "price",
    F.when(F.col("price").isNull(), None)
     .otherwise(F.col("price").cast(DecimalType(20, 0)))
)

bronze_mubawab = bronze_mubawab.withColumn(
    "price",
    F.when(F.col("price").isNull(), None)
     .otherwise(F.col("price").cast(DecimalType(20, 0)))
)

bronze_avito.select("price").show(5, truncate=False)
bronze_mubawab.select("price").show(5, truncate=False)


+-------+
|price  |
+-------+
|2335000|
|1700000|
|1500000|
|NULL   |
|1540000|
+-------+
only showing top 5 rows

+------+
|price |
+------+
|800000|
|480000|
|NULL  |
|NULL  |
|NULL  |
+------+
only showing top 5 rows



In [83]:
# %%
# Avito: tu avais choisi de drop 'location'
bronze_avito.select("location").show(5, truncate=False)

bronze_avito = bronze_avito.drop("location")

+--------+
|location|
+--------+
|        |
|        |
|        |
|        |
|        |
+--------+
only showing top 5 rows



In [84]:
# %%
# Mubawab: normalisation des espaces

bronze_mubawab.select("location").show(10, truncate=False)

bronze_mubawab = bronze_mubawab.withColumn(
    "location",
    F.trim(
        F.regexp_replace(
            F.col("location"),
            r"\s+",
            " "
        )
    )
)

bronze_mubawab.select("location").show(10, truncate=False)

+-----------------------------------------+
|location                                 |
+-----------------------------------------+
|Route de Fez à\n\t\t\t\t\t\t\tMarrakech  |
|Medina à\n\t\t\t\t\t\t\tEssaouira        |
|Marjane à\n\t\t\t\t\t\t\tMeknes          |
|Aghbal                                   |
|Sidi Bennour                             |
|Riad Oulad Mtaa à\n\t\t\t\t\t\t\tTemara  |
|Hay Bensouda à\n\t\t\t\t\t\t\tFès        |
|Plage rose marée à\n\t\t\t\t\t\t\tSkhirat|
|Route Amizmiz à\n\t\t\t\t\t\t\tMarrakech |
|Sidi Bennour                             |
+-----------------------------------------+
only showing top 10 rows

+--------------------------+
|location                  |
+--------------------------+
|Route de Fez à Marrakech  |
|Medina à Essaouira        |
|Marjane à Meknes          |
|Aghbal                    |
|Sidi Bennour              |
|Riad Oulad Mtaa à Temara  |
|Hay Bensouda à Fès        |
|Plage rose marée à Skhirat|
|Route Amizmiz à Marrakech |
|Sidi Benn

In [85]:
# %%
bronze_avito.select("breadcrumbs").show(5, truncate=False)
bronze_avito.select("breadcrumbs_list").show(5, truncate=False)

bronze_mubawab.select("breadcrumbs").show(5, truncate=False)
bronze_mubawab.select("breadcrumbs_list").show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|breadcrumbs                                                                                                                                               |
+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|Accueil > Tout le Maroc > Casablanca > 2 Mars > Avito Immobilier > Ventes Immobilières > Appartements > 152565-Vente Appt à Casablanca Hopitaux de 144 m² |
|Accueil > Tout le Maroc > Casablanca > 2 Mars > Avito Immobilier > Ventes Immobilières > Appartements > APPARTEMENTS NEUFS À VENDRE 2 MARS CASABLANCA     |
|Accueil > Tout le Maroc > Casablanca > 2 Mars > Avito Immobilier > Ventes Immobilières > Appartements > Appart 81 m à vendre à bd anoual neuf             |
|Accueil > Tout le Maroc > Casablanca > 2 Mars > Avito Imm

In [86]:
# %%
# On garde breadcrumbs_list (JSON string) et on drop 'breadcrumbs'

bronze_avito = bronze_avito.drop("breadcrumbs")
bronze_mubawab = bronze_mubawab.drop("breadcrumbs")

In [87]:
# %%
bronze_avito.select("seller_name").show(5, truncate=False)
bronze_mubawab.select("seller_name").show(5, truncate=False)

bronze_avito.groupBy("seller_is_store").count().orderBy("count", ascending=False).show(truncate=False)
bronze_mubawab.groupBy("seller_is_store").count().orderBy("count", ascending=False).show(truncate=False)


+-------------------+
|seller_name        |
+-------------------+
|Yakeey             |
|Faycal K           |
|New Adresse SARL   |
|Mohamed rachid immo|
|Ouchra Immobilier  |
+-------------------+
only showing top 5 rows

+--------------------------+
|seller_name               |
+--------------------------+
|                          |
|Green Property Development|
|                          |
|                          |
|                          |
+--------------------------+
only showing top 5 rows

+---------------+-----+
|seller_is_store|count|
+---------------+-----+
|false          |2797 |
+---------------+-----+

+---------------+-----+
|seller_is_store|count|
+---------------+-----+
|false          |1139 |
|true           |36   |
+---------------+-----+



In [88]:
# %%
# Si tu ne veux pas garder le flag booléen en bronze

bronze_avito = bronze_avito.drop("seller_is_store")
bronze_mubawab = bronze_mubawab.drop("seller_is_store")

In [89]:
# %%
bronze_avito = bronze_avito.drop("scraping_time")
bronze_mubawab = bronze_mubawab.drop("scraping_time")

bronze_avito.select("published_date", "ingest_ts").show(5, truncate=False)
bronze_mubawab.select("published_date", "ingest_ts").show(5, truncate=False)

+------------------------+--------------------------+
|published_date          |ingest_ts                 |
+------------------------+--------------------------+
|2025-11-25T10:00:50.000Z|2025-11-25T14:22:22.511089|
|2025-11-24T16:29:38.000Z|2025-11-25T14:22:22.511089|
|2025-11-24T19:30:23.000Z|2025-11-25T14:22:22.511089|
|2025-11-25T12:56:58.000Z|2025-11-25T14:22:22.511089|
|2025-11-25T14:22:09.000Z|2025-11-25T14:22:22.511089|
+------------------------+--------------------------+
only showing top 5 rows

+-------------------+--------------------------+
|published_date     |ingest_ts                 |
+-------------------+--------------------------+
|2025-11-25 16:10:14|2025-11-25T14:52:56.034717|
|2025-11-25 16:10:19|2025-11-25T14:52:56.034717|
|2025-11-25 16:10:24|2025-11-25T14:52:56.034717|
|2025-11-25 16:10:29|2025-11-25T14:52:56.034717|
|2025-11-25 16:10:33|2025-11-25T14:52:56.034717|
+-------------------+--------------------------+
only showing top 5 rows



In [90]:
# %%
# Parsing générique des timestamps

bronze_avito = (
    bronze_avito
    .withColumn("published_date", F.to_timestamp("published_date"))
    .withColumn("ingest_ts", F.to_timestamp("ingest_ts"))
)

bronze_mubawab = (
    bronze_mubawab
    .withColumn("published_date", F.to_timestamp("published_date"))
    .withColumn("ingest_ts", F.to_timestamp("ingest_ts"))
)

bronze_avito.printSchema()
bronze_mubawab.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- source_site: string (nullable = true)
 |-- offre: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- published_date: timestamp (nullable = true)
 |-- ingest_ts: timestamp (nullable = true)
 |-- category_label: string (nullable = true)
 |-- breadcrumbs_list: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- images: string (nullable = true)
 |-- equipments: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- seller_url: string (nullable = true)
 |-- price: decimal(20,0) (nullable = true)

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- source_site: string (nullable = true)
 |-- offre: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location: string (nullable = true)
 |-- published_date: timestamp (nullable = true)
 |--

In [65]:
# %%
bronze_avito.limit(2).toPandas()

,id,url,source_site,offre,title,description,published_date,ingest_ts,category_label,breadcrumbs_list,attributes,images,equipments,seller_name,seller_url,price
0,57077682,https://www.avito.ma/fr/2_mars/appartements/15...,avito,vente,152565-Vente Appt à Casablanca Hopitaux de 144 m²,À vendre : superbe appartement de 144 m² situé...,2025-11-25 10:00:50,2025-11-25 14:22:22.511089,"Appartements, à vendre","[""Accueil"", ""Tout le Maroc"", ""Casablanca"", ""2 ...","{""Chambres"": ""2"", ""Salle de bain"": ""2"", ""Surfa...",https://content.avito.ma/classifieds/images/10...,Ascenseur; Balcon; Chauffage; Cuisine équipée;...,Yakeey,https://www.avito.ma/fr/boutique/yakeey?id=6359,2335000
1,56615099,https://www.avito.ma/fr/2_mars/appartements/AP...,avito,vente,APPARTEMENTS NEUFS À VENDRE 2 MARS CASABLANCA,Découvrez notre sélection d’appartements haut ...,2025-11-24 16:29:38,2025-11-25 14:22:22.511089,"Appartements, à vendre","[""Accueil"", ""Tout le Maroc"", ""Casablanca"", ""2 ...","{""Chambres"": ""3"", ""Salle de bain"": ""2"", ""Condi...",https://content.avito.ma/classifieds/images/10...,Ascenseur; Balcon; Chauffage; Climatisation; C...,Faycal K,https://www.avito.ma,1700000


In [66]:
bronze_mubawab.limit(2).toPandas()

,id,url,source_site,offre,title,description,location,published_date,ingest_ts,category_label,breadcrumbs_list,attributes,images,equipments,seller_name,seller_url,price
0,8255334,https://www.mubawab.ma/fr/a/8255334/appartemen...,mubawab,vente,Appartement à Vendre 2 chambres proche de lycé...,Votre Agent immobilier vous propose à la vent...,Guéliz à Marrakech,2025-11-25 15:39:26,2025-11-25 14:52:56.034717,"Appartement, à vendre","[""Immobilier Maroc"", ""Immobilier Marrakech"", ""...","{""Type de bien"": ""Appartement"", ""Etat"": ""Bon é...",https://www.mubawab-media.com/ad/8/255/334F/h/...,Terrasse; Ascenseur; Concierge; Salon européen...,VOTRE AGENT IMMOBILIER,https://www.mubawab.ma/fr/b/8392/votre-agent-i...,1850000
1,8255335,https://www.mubawab.ma/fr/a/8255335/vente-terr...,mubawab,vente,Vente terrain à Mhgorgha. \nSurface de 144 m²,Nombreuses possibilités pour ce terrain à la v...,Malabata à Tanger,2025-11-25 15:39:30,2025-11-25 14:52:56.034717,Terrain,"[""Immobilier Maroc"", ""Immobilier Tanger"", ""Ter...","{""Type de bien"": ""Terrain"", ""Type de terrain"":...",https://www.mubawab-media.com/ad/8/255/335F/h/...,,,,None


In [91]:
# %%
from pyspark.sql import functions as F

# 1. Add ingest_date column
bronze_listings = bronze_listings.withColumn("ingest_date", F.to_date("ingest_ts"))

# 2. Collect distinct partitions
partitions = (
    bronze_listings
    .select("source_site", "offre", "ingest_date")
    .distinct()
    .collect()
)

print("Partitions to write:")
for p in partitions:
    print(p)

# 3. Manual writing loop
for p in partitions:
    site = p["source_site"]
    offer = p["offre"]
    date_str = p["ingest_date"].strftime("%Y-%m-%d")

    output_path = f"{bronze_base}/{site}/{offer}/{date_str}"

    print("➡️ Writing:", output_path)

    (
        bronze_listings
        .filter(
            (F.col("source_site") == site) &
            (F.col("offre") == offer) &
            (F.to_date("ingest_date") == date_str)
        )
        .write
        .mode("overwrite")
        .parquet(output_path)
    )

print("✅ Bronze written with clean folder structure")


Partitions to write:
Row(source_site='avito', offre='vente', ingest_date=datetime.date(2025, 11, 25))
Row(source_site='mubawab', offre='vente', ingest_date=datetime.date(2025, 11, 25))
➡️ Writing: abfss://realestate@strealestatehamza.dfs.core.windows.net/bronze/avito/vente/2025-11-25
➡️ Writing: abfss://realestate@strealestatehamza.dfs.core.windows.net/bronze/mubawab/vente/2025-11-25
✅ Bronze written with clean folder structure


In [93]:
bronze_avito.filter(bronze_avito.offre == "vente").count()

2797

In [94]:
from pyspark.sql import functions as F

(
    bronze_avito
    .filter(F.col("offre") == "vente")
    .groupBy(F.to_date("ingest_ts").alias("ingest_date"))
    .count()
    .orderBy("ingest_date")
    .show(truncate=False)
)


+-----------+-----+
|ingest_date|count|
+-----------+-----+
|2025-11-25 |2797 |
+-----------+-----+



In [96]:
# %%
# 0. Imports & Spark session

import os

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import DecimalType
from pyspark.sql.utils import AnalysisException

from dotenv import load_dotenv

# --------------------------------------------------
# 1. Spark + ADLS config
# --------------------------------------------------
load_dotenv()

storage_account = "strealestatehamza"
container = "realestate"

adls_key = os.getenv("ADLS_ACCOUNT_KEY")
if not adls_key:
    raise RuntimeError("ADLS_ACCOUNT_KEY missing from .env")

spark = (
    SparkSession.builder
    .appName("Bronze_Avito_Mubawab")
    .getOrCreate()
)

spark.conf.set(
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
    adls_key,
)
spark.conf.set(
    f"fs.azure.account.key.{storage_account}.blob.core.windows.net",
    adls_key,
)

raw_base = f"abfss://{container}@{storage_account}.dfs.core.windows.net/raw"
bronze_base = f"abfss://{container}@{storage_account}.dfs.core.windows.net/bronze"

print("Spark started ✅")
print("RAW base   :", raw_base)
print("BRONZE base:", bronze_base)


# --------------------------------------------------
# 2. Helper: read parquet if exists
# --------------------------------------------------
def read_parquet_if_exists(path_pattern: str):
    """
    Try to read parquet for a glob pattern.
    Return a DataFrame or None if nothing exists.
    """
    try:
        df = spark.read.parquet(path_pattern)
        print(f"📂 Reading: {path_pattern}")
        return df
    except AnalysisException as e:
        msg = str(e)
        if "Path does not exist" in msg or "PATH_NOT_FOUND" in msg:
            print(f"⚠️ No data found for: {path_pattern}")
            return None
        raise


# --------------------------------------------------
# 3. Read RAW Avito
# --------------------------------------------------
avito_ventes_path    = f"{raw_base}/avito/ventes/*/*/*/*/page=*"
avito_locations_path = f"{raw_base}/avito/locations/*/*/*/*/page=*"

avito_ventes_df    = read_parquet_if_exists(avito_ventes_path)
avito_locations_df = read_parquet_if_exists(avito_locations_path)

avito_dfs = [df for df in [avito_ventes_df, avito_locations_df] if df is not None]
if not avito_dfs:
    raise RuntimeError("No Avito raw data found (ventes or locations).")

if len(avito_dfs) == 1:
    df_avito = avito_dfs[0]
else:
    df_avito = avito_dfs[0].unionByName(avito_dfs[1])

print("✅ RAW Avito loaded")
df_avito.printSchema()


# --------------------------------------------------
# 4. Read RAW Mubawab
# --------------------------------------------------
mubawab_ventes_path    = f"{raw_base}/mubawab/ventes/*/*/*/*/page=*"
mubawab_locations_path = f"{raw_base}/mubawab/locations/*/*/*/*/page=*"

mubawab_ventes_df    = read_parquet_if_exists(mubawab_ventes_path)
mubawab_locations_df = read_parquet_if_exists(mubawab_locations_path)

mubawab_dfs = [df for df in [mubawab_ventes_df, mubawab_locations_df] if df is not None]
if not mubawab_dfs:
    raise RuntimeError("No Mubawab raw data found (ventes or locations).")

if len(mubawab_dfs) == 1:
    df_mubawab = mubawab_dfs[0]
else:
    df_mubawab = mubawab_dfs[0].unionByName(mubawab_dfs[1])

print("✅ RAW Mubawab loaded")
df_mubawab.printSchema()


# --------------------------------------------------
# 5. BRONZE Avito (column selection)
# --------------------------------------------------
bronze_avito = df_avito.select(
    # Identifiers
    "id",
    "url",
    "source_site",
    "offre",

    # Main business content
    "title",
    "description",
    "price_text",
    "location",

    # Dates
    "published_date",
    "scraping_time",
    "ingest_ts",

    # Metadata
    "category_label",
    "breadcrumbs",
    "breadcrumbs_list",
    "attributes",

    # Media
    "images",
    "equipments",

    # Seller info
    "seller_name",
    "seller_url",
    "seller_is_store"
)

print("Schema BRONZE Avito (raw selection):")
bronze_avito.printSchema()


# --------------------------------------------------
# 6. BRONZE Mubawab (column selection)
# --------------------------------------------------
bronze_mubawab = df_mubawab.select(
    # Identifiers
    "id",
    "url",
    "source_site",
    "offre",

    # Main business content
    "title",
    "description",
    "price_text",
    "location",

    # Dates
    "published_date",
    "scraping_time",
    "ingest_ts",

    # Metadata
    "category_label",
    "breadcrumbs",
    "breadcrumbs_list",
    "attributes",

    # Media
    "images",
    "equipments",

    # Seller info
    "seller_name",
    "seller_url",
    "seller_is_store"
)

print("Schema BRONZE Mubawab (raw selection):")
bronze_mubawab.printSchema()


# --------------------------------------------------
# 7. Basic QC: NULL id/url
# --------------------------------------------------
null_count_avito = bronze_avito.filter(
    (F.col("id").isNull()) | (F.col("url").isNull())
).count()

null_count_mubawab = bronze_mubawab.filter(
    (F.col("id").isNull()) | (F.col("url").isNull())
).count()

print("NULL id/url Avito  :", null_count_avito)
print("NULL id/url Mubawab:", null_count_mubawab)


# --------------------------------------------------
# 8. Deduplicate on URL (keep earliest scraping_time)
# --------------------------------------------------
w_avito = Window.partitionBy("url").orderBy(F.col("scraping_time").asc())
bronze_avito = (
    bronze_avito
    .withColumn("rn", F.row_number().over(w_avito))
    .filter("rn = 1")
    .drop("rn")
)

w_mubawab = Window.partitionBy("url").orderBy(F.col("scraping_time").asc())
bronze_mubawab = (
    bronze_mubawab
    .withColumn("rn", F.row_number().over(w_mubawab))
    .filter("rn = 1")
    .drop("rn")
)

print("✅ Dedup done on URL")


# --------------------------------------------------
# 9. Check offre distribution
# --------------------------------------------------
print("Offre distribution Avito:")
bronze_avito.groupBy("offre").count().orderBy("count", ascending=False).show(truncate=False)

print("Offre distribution Mubawab:")
bronze_mubawab.groupBy("offre").count().orderBy("count", ascending=False).show(truncate=False)


# --------------------------------------------------
# 10. price_text -> price (double -> DECIMAL(20,0))
# --------------------------------------------------
def normalize_price(df):
    df = (
        df
        .withColumn(
            "price",
            F.when(
                (F.col("price_text").isNull()) |
                (F.lower(F.col("price_text")) == "null") |
                (F.trim(F.col("price_text")) == ""),
                None
            ).otherwise(
                F.regexp_replace(
                    F.regexp_replace(
                        F.col("price_text"),
                        r"(?i)\s*dh\s*", ""     # remove DH / dh
                    ),
                    r"[^\d]", ""               # keep only digits
                ).cast("double")
            )
        )
        .drop("price_text")
    )

    df = df.withColumn(
        "price",
        F.when(F.col("price").isNull(), None)
         .otherwise(F.col("price").cast(DecimalType(20, 0)))
    )

    return df

bronze_avito   = normalize_price(bronze_avito)
bronze_mubawab = normalize_price(bronze_mubawab)

print("Sample prices Avito:")
bronze_avito.select("price").show(5, truncate=False)

print("Sample prices Mubawab:")
bronze_mubawab.select("price").show(5, truncate=False)


# --------------------------------------------------
# 11. Locations & breadcrumbs tweaks
# --------------------------------------------------
# Avito: drop location (as per your previous choice)
print("Avito location sample:")
bronze_avito.select("location").show(5, truncate=False)
bronze_avito = bronze_avito.drop("location")

# Mubawab: normalize spaces in location
print("Mubawab location BEFORE:")
bronze_mubawab.select("location").show(5, truncate=False)

bronze_mubawab = bronze_mubawab.withColumn(
    "location",
    F.trim(
        F.regexp_replace(
            F.col("location"),
            r"\s+",
            " "
        )
    )
)

print("Mubawab location AFTER:")
bronze_mubawab.select("location").show(5, truncate=False)

# Keep breadcrumbs_list (JSON string) and drop breadcrumbs
bronze_avito   = bronze_avito.drop("breadcrumbs")
bronze_mubawab = bronze_mubawab.drop("breadcrumbs")


# --------------------------------------------------
# 12. Seller & dates cleanup
# --------------------------------------------------
bronze_avito   = bronze_avito.drop("seller_is_store")
bronze_mubawab = bronze_mubawab.drop("seller_is_store")

bronze_avito   = bronze_avito.drop("scraping_time")
bronze_mubawab = bronze_mubawab.drop("scraping_time")

print("Dates sample Avito:")
bronze_avito.select("published_date", "ingest_ts").show(5, truncate=False)

print("Dates sample Mubawab:")
bronze_mubawab.select("published_date", "ingest_ts").show(5, truncate=False)

# Parse timestamps
bronze_avito = (
    bronze_avito
    .withColumn("published_date", F.to_timestamp("published_date"))
    .withColumn("ingest_ts", F.to_timestamp("ingest_ts"))
)

bronze_mubawab = (
    bronze_mubawab
    .withColumn("published_date", F.to_timestamp("published_date"))
    .withColumn("ingest_ts", F.to_timestamp("ingest_ts"))
)

print("Schemas after timestamp parsing:")
bronze_avito.printSchema()
bronze_mubawab.printSchema()

# --------------------------------------------------
# 13. Align schemas before union
# --------------------------------------------------

# Avito currently has NO `location` column (we dropped it earlier)
# → we add it back as NULL so it matches Mubawab schema
if "location" not in bronze_avito.columns:
    bronze_avito = bronze_avito.withColumn("location", F.lit(None).cast("string"))

# Reorder columns to ensure same order in both DFs (not strictly needed with unionByName,
# but it's nice & explicit)
ordered_cols = [
    "id",
    "url",
    "source_site",
    "offre",
    "title",
    "description",
    "location",          # <-- now present in both
    "published_date",
    "ingest_ts",
    "category_label",
    "breadcrumbs_list",
    "attributes",
    "images",
    "equipments",
    "seller_name",
    "seller_url",
    "price",
]

bronze_avito   = bronze_avito.select(ordered_cols)
bronze_mubawab = bronze_mubawab.select(ordered_cols)

# Now union with allowMissingColumns just in case future schema evolves
bronze_listings = bronze_avito.unionByName(bronze_mubawab, allowMissingColumns=True)

print("✅ Unified BRONZE listings created")
print("Total rows BRONZE listings:", bronze_listings.count())

print("Rows per (source_site, offre):")
bronze_listings.groupBy("source_site", "offre").count().orderBy("source_site", "offre").show(truncate=False)
# --------------------------------------------------
# 14. Add ingest_date (partition column)
# --------------------------------------------------
bronze_listings = bronze_listings.withColumn("ingest_date", F.to_date("ingest_ts"))

print("Rows per (source_site, offre, ingest_date):")
bronze_listings.groupBy("source_site", "offre", "ingest_date") \
    .count() \
    .orderBy("source_site", "offre", "ingest_date") \
    .show(truncate=False)

# --------------------------------------------------
# 15. Manual partition write
# --------------------------------------------------
partitions = (
    bronze_listings
    .select("source_site", "offre", "ingest_date")
    .distinct()
    .collect()
)

print("Partitions to write:")
for p in partitions:
    print(p)

for p in partitions:
    site = p["source_site"]
    offer = p["offre"]
    ingest_date = p["ingest_date"]          # Python date
    date_str = ingest_date.strftime("%Y-%m-%d")

    output_path = f"{bronze_base}/{site}/{offer}/{date_str}"

    df_part = bronze_listings.filter(
        (F.col("source_site") == site) &
        (F.col("offre") == offer) &
        (F.col("ingest_date") == F.lit(ingest_date))
    )

    part_count = df_part.count()
    print(f"➡️ Writing: {output_path}  (rows={part_count})")

    df_part.write.mode("overwrite").parquet(output_path)

print("✅ Bronze written with clean folder structure")

# Sanity: Avito ventes row count in BRONZE DF
avito_vente_total = bronze_avito.filter(F.col("offre") == "vente").count()
print("DEBUG - bronze_avito.offre == 'vente' rows:", avito_vente_total)


Spark started ✅
RAW base   : abfss://realestate@strealestatehamza.dfs.core.windows.net/raw
BRONZE base: abfss://realestate@strealestatehamza.dfs.core.windows.net/bronze
📂 Reading: abfss://realestate@strealestatehamza.dfs.core.windows.net/raw/avito/ventes/*/*/*/*/page=*
📂 Reading: abfss://realestate@strealestatehamza.dfs.core.windows.net/raw/avito/locations/*/*/*/*/page=*
✅ RAW Avito loaded
root
 |-- attributes: string (nullable = true)
 |-- breadcrumbs: string (nullable = true)
 |-- breadcrumbs_list: string (nullable = true)
 |-- category_label: string (nullable = true)
 |-- description: string (nullable = true)
 |-- equipments: string (nullable = true)
 |-- id: string (nullable = true)
 |-- images: string (nullable = true)
 |-- location: string (nullable = true)
 |-- price_text: string (nullable = true)
 |-- published_date: string (nullable = true)
 |-- scraping_time: string (nullable = true)
 |-- seller_is_store: boolean (nullable = true)
 |-- seller_name: string (nullable = true)
 |